##  Import Libraries

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import random
import math

##  Initialize MediaPipe

In [6]:

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils


## Video capture

In [8]:

cap = cv2.VideoCapture(0)


## Game state

In [13]:

snake = []
snake_length = 0
score = 0
food = (random.randint(100, 500), random.randint(100, 400))
food_emoji = random.choice(["🍎", "🍇", "🍌"])

font = cv2.FONT_HERSHEY_SIMPLEX

##  Collision with Food

In [16]:
def get_index_finger_position(frame, results):
    h, w, c = frame.shape
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm = handLms.landmark[8]  # Index fingertip
            return int(lm.x * w), int(lm.y * h)
    return None

## Draw Snake and Food

In [21]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    index_finger = get_index_finger_position(frame, results)
    if index_finger:
        snake.append(index_finger)

        # Trim the snake to the correct length
        if len(snake) > snake_length:
            snake.pop(0)

        # Check collision with food
        if math.hypot(index_finger[0] - food[0], index_finger[1] - food[1]) < 30:
            score += 1
            snake_length += 10
            food = (random.randint(100, 500), random.randint(100, 400))
            food_emoji = random.choice(["🍎", "🍇", "🍌"])

    # Draw snake
    for i in range(1, len(snake)):
        cv2.line(frame, snake[i-1], snake[i], (0, 255, 0), 15)

    # Draw food (as circle because OpenCV can't draw emojis)
    cv2.circle(frame, food, 20, (0, 0, 255), -1)

    # Display score
    cv2.putText(frame, f"Score: {score}", (10, 40), font, 1, (255, 255, 255), 2)

    cv2.imshow("Hand-Controlled Snake Game", frame)

    # Break loop on ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

## Display Score

In [18]:
print(f"Game Over! Your Final Score is: {score}")

Game Over! Your Final Score is: 12
